In [ ]:
x_train = df[WQP].dropna()
print(WQP)

In [ ]:
df_results = pd.DataFrame()

In [ ]:
# convert an array of values into a dataset matrix
def create_dataset(values, look_back):
    np.random.seed(42)
     # set empty lists 
    _x, _y = [], []
    for i in range(len(values)-look_back-1):
        a = values[i:(i+look_back)]      # stack a list of values
        _x.append(a)                        # set x
        _y.append(values[i + look_back]) # set y
    return np.array(_x), np.array(_y)

In [ ]:
num = 2
values = df[WQP].values
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(values[~np.isnan(values)].reshape(-1, 1))

In [ ]:
# split into train and test sets
train_size = int(len(dataset) * split)
test_size = len(dataset) - train_size
train, test = dataset[:train_size], dataset[train_size:]
# reshape into X=t and Y=t+1
#look_back = 20
train_x, train_y = create_dataset(train, look_back)
test_x, test_y = create_dataset(test, look_back)
# reshape input to be [samples, time steps, features]
train_x = np.reshape(train_x, (train_x.shape[0], train_x.shape[1], 1))
test_x = np.reshape(test_x, (test_x.shape[0], test_x.shape[1], 1))

In [ ]:
Lgr = test_size-wz-1
dLgr = train_size+wz+1

In [ ]:
   def scatter_plot(y_true, y_pred, model_name):
    plt.figure(figsize = (10, 10))
    plt.scatter(y_true, y_pred)   
    p1 = max(max(y_pred), max(y_true))
    p2 = min(min(y_pred), min(y_pred))
    plt.plot([p1, p2], [p1, p2], 'b-')
    plt.xlabel('Observed ' + WQPu, fontsize=15, color='black')
    plt.ylabel('Predicted ' + WQPu, fontsize=15, color='black')
    plt.axis('equal')
    plt.grid(linestyle=':', linewidth=1)
    plt.savefig('n/'+ WQPn +'_SP_'+ model_name + l + s + e + w + b + o + d +'.pdf', dpi=300)
    plt.close()

In [ ]:
def metrics_time_series(y_true, y_pred):
    mae = round         (mean_absolute_error(y_true, y_pred), 7)
    mse = round         (mean_squared_error(y_true, y_pred), 7)
    rmse= round         (np.sqrt(mean_squared_error(y_true, y_pred)), 7)
    mape= round         (mean_absolute_percentage_error(y_true, y_pred), 7)
   
    return mae, mse, rmse, mape 

In [ ]:
def plotting_predictions(dataset, look_back, train_predict,  test_predict,DLname):
    np.random.seed(42)
    # shift train predictions for plotting
    trainPredictPlot = np.empty_like(dataset)
    trainPredictPlot[:, :] = np.nan
    trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
    # shift test predictions for plotting
    testPredictPlot = np.empty_like(dataset)
    testPredictPlot[:, :] = np.nan
    testPredictPlot[len(train_predict)+(look_back*2)+1:len(dataset)-1, :] = test_predict
    # plot baseline and predictions
    plt.figure(figsize=(10,6))
    plt.plot(scaler.inverse_transform(dataset[wz:]), 'b', label="True Data")
    plt.plot(trainPredictPlot[wz:], 'r', label='Train')
    plt.plot(testPredictPlot[wz:], 'g', label="Prediction")
    plt.ylabel(WQPu, fontsize=15)
    plt.xlabel('Days', fontsize=15)
    plt.grid(linestyle=':', linewidth=1)
    plt.legend()
    #plt.savefig("'1'+model.pdf", dpi=300)
    plt.title('True future vs prediction for ' + DLname)
    plt.savefig('n/'+ WQPn +'_pred'+ DLname + l + s + e + w + b + o + d +'.pdf', dpi=300)
    plt.close()
    return    

In [ ]:
def plot_loss (model_name):
    fig = plt.figure(figsize = (10, 6))
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Train vs Validation Loss for ' + model_name)
    plt.ylabel('Loss', fontsize=15)
    plt.xlabel('epoch', fontsize=15)
    plt.legend(['Train loss', 'Validation loss'], loc='upper right')
    plt.grid(linestyle=':', linewidth=1)
    plt.savefig('n/'+ WQPn +'_loss_'+ model_name + l + s + e + w + b + o + d +'.pdf', dpi=300)
    plt.close()

In [ ]:
def time_series_deep_learning1(x_train, y_train, x_test, y_test):
    tf.keras.backend.clear_session()
    tf.random.set_seed(51)
    name = "LSTM"
    # instance of a Sequential model
    ST = time.time()

    model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(32, input_shape=(look_back, 1)),
    tf.keras.layers.Dense(30, activation="relu"),
    tf.keras.layers.Dense(10, activation="relu"),
    tf.keras.layers.Dense(1),
    tf.keras.layers.Lambda(lambda x: x * 400)  
      ])
      
    model.compile(loss='mean_squared_error', optimizer= opt, metrics=['accuracy'])
    history=model.fit(x_train, y_train, epochs=epno, validation_split = 0.2, batch_size=bz, verbose=0)
    ET = time.time()
    train_time = ET - ST

    # make predictions
    train_predict = model.predict(x_train)
    
    ST = time.time()
    test_predict = model.predict(x_test)
    ET = time.time()
    test_time = ET - ST
    
   ##model.summary ()
    
    # invert predictions
    train_predict = scaler.inverse_transform(train_predict)
    train_y = scaler.inverse_transform(y_train)
    test_predict = scaler.inverse_transform(test_predict)
    test_y = scaler.inverse_transform(y_test)    

    # compute the metrics 
    #print("Train")
    mae_train, mse_train, rmse_train, mape_train = metrics_time_series( train_y, train_predict)
    #print("Test")
    mae_test, mse_test, rmse_test, mape_test = metrics_time_series( test_y, test_predict)
      
    return history,train_predict, train_y, test_predict, test_y, name, pd.DataFrame([name, mae_train, mse_train, rmse_train, mape_train, train_time, mae_test, mse_test, rmse_test, mape_test,test_time], index= ["Name", "mae_train", "mse_train","rmse_train", "mape_train", "Train_time_s","mae_test", "mse_test", "rmse_test", "mape_test", "Test_time_s"]).T

    
history, x_train_predict_lstm, y_train_lstm,x_test_predict_lstm, y_test_lstm, name, res= time_series_deep_learning1(train_x, train_y, test_x, test_y)
plot_loss (name)
df_results = df_results.append(res)

In [ ]:
def time_series_deep_learning2(x_train, y_train, x_test, y_test):
    tf.keras.backend.clear_session()
    tf.random.set_seed(51)
    name = "GRU"
    # instance of a Sequential model
    ST = time.time()
    
    model = tf.keras.models.Sequential([
    tf.keras.layers.GRU(32, input_shape=(look_back, 1)),
    tf.keras.layers.Dense(30, activation="relu"),
    tf.keras.layers.Dense(10, activation="relu"),
    tf.keras.layers.Dense(1),
    tf.keras.layers.Lambda(lambda x: x * 400)
    ])
    
    model.compile(loss='mean_squared_error', optimizer= opt, metrics=['accuracy'])
    history=model.fit(x_train, y_train, epochs=epno, validation_split = 0.2, batch_size=bz, verbose=0)
    ET = time.time()
    train_time = ET - ST

    # make predictions
    train_predict = model.predict(x_train)
    
    ST = time.time()
    test_predict = model.predict(x_test)
    ET = time.time()
    test_time = ET - ST
    
   #model.summary ()
    
    # invert predictions
    train_predict = scaler.inverse_transform(train_predict)
    train_y = scaler.inverse_transform(y_train)
    test_predict = scaler.inverse_transform(test_predict)
    test_y = scaler.inverse_transform(y_test)    

    # compute the metrics 
    #print("Train")
    mae_train, mse_train, rmse_train, mape_train = metrics_time_series( train_y, train_predict)
    #print("Test")
    mae_test, mse_test, rmse_test, mape_test = metrics_time_series( test_y, test_predict)
      
    return history,train_predict, train_y, test_predict, test_y, name, pd.DataFrame([name, mae_train, mse_train, rmse_train, mape_train, train_time, mae_test, mse_test, rmse_test, mape_test,test_time], index= ["Name", "mae_train", "mse_train","rmse_train", "mape_train", "Train_time_s","mae_test", "mse_test", "rmse_test", "mape_test", "Test_time_s"]).T

history, x_train_predict_gru, y_train_gru, x_test_predict_gru, y_test_gru, name, res= time_series_deep_learning2(train_x, train_y, test_x, test_y)
plot_loss (name)
df_results = df_results.append(res)

In [ ]:
from keras.layers import *
from keras.models import *
from keras import backend as K

In [ ]:
# Add attention layer to the deep learning network
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name='attention_weight', shape=(input_shape[-1],1), initializer='random_normal', trainable=True)
        self.b=self.add_weight(name='attention_bias', shape=(input_shape[1],1), initializer='zeros', trainable=True)        
        super(attention, self).build(input_shape)

    def call(self,x):
        # Alignment scores. Pass them through tanh function
        e = K.tanh(K.dot(x,self.W)+self.b)
        # Remove dimension of size 1
        e = K.squeeze(e, axis=-1)   
        # Compute the weights
        alpha = K.softmax(e)
        # Reshape to tensorFlow format
        alpha = K.expand_dims(alpha, axis=-1)
        # Compute the context vector
        context = x * alpha
        context = K.sum(context, axis=1)
        return context

In [ ]:
def time_series_deep_learningBiLSTM(x_train, y_train, x_test, y_test):
    tf.keras.backend.clear_session()
    tf.random.set_seed(51)
    name = "BiLSTM"
    ST = time.time()
    
    model = tf.keras.models.Sequential([
    tf.keras.layers.Bidirectional(LSTM(32,input_shape=(look_back, 1))),
    tf.keras.layers.Dense(30, activation="relu"),
    tf.keras.layers.Dense(10, activation="relu"),
    tf.keras.layers.Dense(1),
    tf.keras.layers.Lambda(lambda x: x * 400)  
     ])
      
    model.compile(loss='mean_squared_error', optimizer= opt, metrics=['accuracy'])
    history=model.fit(x_train, y_train, epochs=epno, validation_split = 0.2, batch_size=bz, verbose=0)
    ET = time.time()
    train_time = ET - ST

    # make predictions
    train_predict = model.predict(x_train)
    
    ST = time.time()
    test_predict = model.predict(x_test)
    ET = time.time()
    test_time = ET - ST
    
    #model.summary ()
    
    # invert predictions
    train_predict = scaler.inverse_transform(train_predict)
    train_y = scaler.inverse_transform(y_train)
    test_predict = scaler.inverse_transform(test_predict)
    test_y = scaler.inverse_transform(y_test)    

    # compute the metrics 
    #print("Train")
    mae_train, mse_train, rmse_train, mape_train = metrics_time_series( train_y, train_predict)
    #print("Test")
    mae_test, mse_test, rmse_test, mape_test = metrics_time_series( test_y, test_predict)
      
    return history,train_predict, train_y, test_predict, test_y, name, pd.DataFrame([name, mae_train, mse_train, rmse_train, mape_train, train_time, mae_test, mse_test, rmse_test, mape_test,test_time], index= ["Name", "mae_train", "mse_train","rmse_train", "mape_train", "Train_time_s","mae_test", "mse_test", "rmse_test", "mape_test", "Test_time_s"]).T

history, x_train_predict_BiLSTM, y_train_BiLSTM,x_test_predict_BiLSTM, y_test_BiLSTM, name, res= time_series_deep_learningBiLSTM(train_x, train_y, test_x, test_y)
plot_loss (name)
df_results = df_results.append(res)

In [ ]:
def time_series_deep_learningBiGRU(x_train, y_train, x_test, y_test):
    tf.keras.backend.clear_session()
    tf.random.set_seed(51)
    name = "BiGRU"
    ST = time.time()
    
    model = tf.keras.models.Sequential([
    tf.keras.layers.Bidirectional(GRU(32, input_shape=(look_back, 1))),
    tf.keras.layers.Dense(30, activation="relu"),
    tf.keras.layers.Dense(10, activation="relu"),
    tf.keras.layers.Dense(1),
    tf.keras.layers.Lambda(lambda x: x * 400)  
     ])
      
    model.compile(loss='mean_squared_error', optimizer= opt, metrics=['accuracy'])
    history=model.fit(x_train, y_train, epochs=epno, validation_split = 0.2, batch_size=bz, verbose=0)
    ET = time.time()
    train_time = ET - ST

    # make predictions
    train_predict = model.predict(x_train)
    
    ST = time.time()
    test_predict = model.predict(x_test)
    ET = time.time()
    test_time = ET - ST
    
    #model.summary ()
    
    # invert predictions
    train_predict = scaler.inverse_transform(train_predict)
    train_y = scaler.inverse_transform(y_train)
    test_predict = scaler.inverse_transform(test_predict)
    test_y = scaler.inverse_transform(y_test)    

    # compute the metrics 
    #print("Train")
    mae_train, mse_train, rmse_train, mape_train = metrics_time_series( train_y, train_predict)
    #print("Test")
    mae_test, mse_test, rmse_test, mape_test = metrics_time_series( test_y, test_predict)
      
    return history,train_predict, train_y, test_predict, test_y, name, pd.DataFrame([name, mae_train, mse_train, rmse_train, mape_train, train_time, mae_test, mse_test, rmse_test, mape_test,test_time], index= ["Name", "mae_train", "mse_train","rmse_train", "mape_train", "Train_time_s","mae_test", "mse_test", "rmse_test", "mape_test", "Test_time_s"]).T

history, x_train_predict_BiGRU, y_train_BiGRU,x_test_predict_BiGRU, y_test_BiGRU, name, res= time_series_deep_learningBiGRU(train_x, train_y, test_x, test_y)
plot_loss (name)
df_results = df_results.append(res)

In [ ]:
def time_series_deep_learning6(x_train, y_train, x_test, y_test):
    tf.keras.backend.clear_session()
    tf.random.set_seed(51)
    name = "LSTM_Att"
    # instance of a Sequential model
    ST = time.time()

    hidden_units = 2
    input_shape=(look_back, 1)
    x=Input(shape=input_shape)

    LSTM_model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(32, return_sequences=True, activation ='relu'),
    tf.keras.layers.Dense(30, activation="relu"),
    tf.keras.layers.Dense(10, activation="relu"),
    tf.keras.layers.Dense(1),
    tf.keras.layers.Lambda(lambda x: x * 400)  
      ])(x)
    
    attention_layer = attention()(LSTM_model)
    outputs=Dense(1, trainable=True, activation='tanh')(attention_layer)
    model=Model(x,outputs)

    model.compile(loss='mean_squared_error', optimizer= opt, metrics=['accuracy'])
    history=model.fit(x_train, y_train, epochs=epno, validation_split = 0.2, batch_size=bz, verbose=0)
    ET = time.time()
    train_time = ET - ST

    # make predictions
    train_predict = model.predict(x_train)
    
    ST = time.time()
    test_predict = model.predict(x_test)
    ET = time.time()
    test_time = ET - ST
    
   ##model.summary ()
    
    # invert predictions
    train_predict = scaler.inverse_transform(train_predict)
    train_y = scaler.inverse_transform(y_train)
    test_predict = scaler.inverse_transform(test_predict)
    test_y = scaler.inverse_transform(y_test)    

    # compute the metrics 
    #print("Train")
    mae_train, mse_train, rmse_train, mape_train = metrics_time_series( train_y, train_predict)
    #print("Test")
    mae_test, mse_test, rmse_test, mape_test = metrics_time_series( test_y, test_predict)
      
    return history,train_predict, train_y, test_predict, test_y, name, pd.DataFrame([name, mae_train, mse_train, rmse_train, mape_train, train_time, mae_test, mse_test, rmse_test, mape_test,test_time], index= ["Name", "mae_train", "mse_train","rmse_train", "mape_train", "Train_time_s","mae_test", "mse_test", "rmse_test", "mape_test", "Test_time_s"]).T
    
history, x_train_predict_lstm_Att, y_train_lstm_Att,x_test_predict_lstm_Att, y_test_lstm_Att, name, res= time_series_deep_learning6(train_x, train_y, test_x, test_y)
plot_loss (name)
df_results = df_results.append(res)

In [ ]:
def time_series_deep_learning7(x_train, y_train, x_test, y_test):
    tf.keras.backend.clear_session()
    tf.random.set_seed(51)
    name = "GRU_Att"
    # instance of a Sequential model
    ST = time.time()

    hidden_units = 2
    input_shape=(look_back, 1)
    x=Input(shape=input_shape)

    GRU_model = tf.keras.models.Sequential([
    tf.keras.layers.GRU(32, return_sequences=True, activation ='relu'),
    tf.keras.layers.Dense(30, activation="relu"),
    tf.keras.layers.Dense(10, activation="relu"),
    tf.keras.layers.Dense(1),
    tf.keras.layers.Lambda(lambda x: x * 400)  
      ])(x)
    
    attention_layer = attention()(GRU_model)
    outputs=Dense(1, trainable=True, activation='tanh')(attention_layer)
    model=Model(x,outputs)

    model.compile(loss='mean_squared_error', optimizer= opt, metrics=['accuracy'])
    history=model.fit(x_train, y_train, epochs=epno, validation_split = 0.2, batch_size=bz, verbose=0)
    ET = time.time()
    train_time = ET - ST
    
    # make predictions
    train_predict = model.predict(x_train)
    
    ST = time.time()
    test_predict = model.predict(x_test)
    ET = time.time()
    test_time = ET - ST
    
   #model.summary ()
    
    # invert predictions
    train_predict = scaler.inverse_transform(train_predict)
    train_y = scaler.inverse_transform(y_train)
    test_predict = scaler.inverse_transform(test_predict)
    test_y = scaler.inverse_transform(y_test)    

    # compute the metrics 
    #print("Train")
    mae_train, mse_train, rmse_train, mape_train = metrics_time_series( train_y, train_predict)
    #print("Test")
    mae_test, mse_test, rmse_test, mape_test = metrics_time_series( test_y, test_predict)
      
    return history,train_predict, train_y, test_predict, test_y, name, pd.DataFrame([name, mae_train, mse_train, rmse_train, mape_train, train_time, mae_test, mse_test, rmse_test, mape_test,test_time], index= ["Name", "mae_train", "mse_train","rmse_train", "mape_train", "Train_time_s","mae_test", "mse_test", "rmse_test", "mape_test", "Test_time_s"]).T

history, x_train_predict_gru_Att, y_train_gru_Att, x_test_predict_gru_Att, y_test_gru_Att, name, res= time_series_deep_learning7(train_x, train_y, test_x, test_y)
plot_loss (name)
df_results = df_results.append(res)

In [ ]:
def time_series_deep_learningBiLSTM_Att(x_train, y_train, x_test, y_test):
    tf.keras.backend.clear_session()
    tf.random.set_seed(51)
    name = "Bi-LSTM_Att"
    # instance of a Sequential model
    ST = time.time()

    hidden_units = 2
    input_shape=(look_back, 1)
    x=Input(shape=input_shape)

    BiLSTM_model = tf.keras.models.Sequential([
    tf.keras.layers.Bidirectional(LSTM(32, return_sequences=True)),
    tf.keras.layers.Dense(30, activation="relu"),
    tf.keras.layers.Dense(10, activation="relu"),
    tf.keras.layers.Dense(1),
   #tf.keras.layers.Lambda(lambda x: x * 400)  
      ])(x)
    
    attention_layer = attention()(BiLSTM_model)
    outputs=Dense(1, trainable=True, activation='tanh')(attention_layer)
    model=Model(x,outputs)

    model.compile(loss='mean_squared_error', optimizer= opt, metrics=['accuracy'])
    history=model.fit(x_train, y_train, epochs=epno, validation_split = 0.2, batch_size=bz, verbose=0)
    ET = time.time()
    train_time = ET - ST

    # make predictions
    train_predict = model.predict(x_train)
    
    ST = time.time()
    test_predict = model.predict(x_test)
    ET = time.time()
    test_time = ET - ST
    
    #model.summary ()
    
    # invert predictions
    train_predict = scaler.inverse_transform(train_predict)
    train_y = scaler.inverse_transform(y_train)
    test_predict = scaler.inverse_transform(test_predict)
    test_y = scaler.inverse_transform(y_test)    

    # compute the metrics 
    #print("Train")
    mae_train, mse_train, rmse_train, mape_train = metrics_time_series( train_y, train_predict)
    #print("Test")
    mae_test, mse_test, rmse_test, mape_test = metrics_time_series( test_y, test_predict)
      
    return history,train_predict, train_y, test_predict, test_y, name, pd.DataFrame([name, mae_train, mse_train, rmse_train, mape_train, train_time, mae_test, mse_test, rmse_test, mape_test,test_time], index= ["Name", "mae_train", "mse_train","rmse_train", "mape_train", "Train_time_s","mae_test", "mse_test", "rmse_test", "mape_test", "Test_time_s"]).T

history, x_train_predict_Bilstm_Att, y_train_Bilstm_Att,x_test_predict_Bilstm_Att, y_test_Bilstm_Att, name, res= time_series_deep_learningBiLSTM_Att(train_x, train_y, test_x, test_y)
plot_loss (name)
df_results = df_results.append(res)    


In [ ]:
def time_series_deep_learningBiGRU_Att(x_train, y_train, x_test, y_test):
    tf.keras.backend.clear_session()
    tf.random.set_seed(51)
    name = "Bi-GRU_Att"
    # instance of a Sequential model
    ST = time.time()

    hidden_units = 2
    input_shape=(look_back, 1)
    x=Input(shape=input_shape)

    BiGRU_model = tf.keras.models.Sequential([
    tf.keras.layers.Bidirectional(GRU(32, return_sequences=True)),
    tf.keras.layers.Dense(30, activation="relu"),
    tf.keras.layers.Dense(10, activation="relu"),
    tf.keras.layers.Dense(1),
   #tf.keras.layers.Lambda(lambda x: x * 400)  
      ])(x)
    
    attention_layer = attention()(BiGRU_model)
    outputs=Dense(1, trainable=True, activation='tanh')(attention_layer)
    model=Model(x,outputs)

    model.compile(loss='mean_squared_error', optimizer= opt, metrics=['accuracy'])
    history=model.fit(x_train, y_train, epochs=epno, validation_split = 0.2, batch_size=bz, verbose=0)
    ET = time.time()
    train_time = ET - ST
    
    # make predictions
    train_predict = model.predict(x_train)
    
    ST = time.time()
    test_predict = model.predict(x_test)
    ET = time.time()
    test_time = ET - ST
    
    #model.summary ()
    
    # invert predictions
    train_predict = scaler.inverse_transform(train_predict)
    train_y = scaler.inverse_transform(y_train)
    test_predict = scaler.inverse_transform(test_predict)
    test_y = scaler.inverse_transform(y_test)    

    # compute the metrics 
    #print("Train")
    mae_train, mse_train, rmse_train, mape_train = metrics_time_series( train_y, train_predict)
    #print("Test")
    mae_test, mse_test, rmse_test, mape_test = metrics_time_series( test_y, test_predict)
      
    return history,train_predict, train_y, test_predict, test_y, name, pd.DataFrame([name, mae_train, mse_train, rmse_train, mape_train, train_time, mae_test, mse_test, rmse_test, mape_test,test_time], index= ["Name", "mae_train", "mse_train","rmse_train", "mape_train", "Train_time_s","mae_test", "mse_test", "rmse_test", "mape_test", "Test_time_s"]).T

history, x_train_predict_Bigru_Att, y_train_Bigru_Att, x_test_predict_Bigru_Att, y_test_Bigru_Att, name, res= time_series_deep_learningBiGRU_Att(train_x, train_y, test_x, test_y)
plot_loss (name)
df_results = df_results.append(res)

In [ ]:
#b : blue.  g : green.  r : red.    c : cyan.   m : magenta.    y : yellow. k : black.  w : white.
#look_back = 20
testPredictPlot11 = np.empty_like(dataset)
testPredictPlot11[:, :] = np.nan
testPredictPlot11[0:Lgr, :] = x_test_predict_lstm
testPredictPlot12 = np.empty_like(dataset)
testPredictPlot12[:, :] = np.nan
testPredictPlot12[0:Lgr, :] = x_test_predict_BiLSTM
testPredictPlot13 = np.empty_like(dataset)
testPredictPlot13[:, :] = np.nan
testPredictPlot13[0:Lgr, :] = x_test_predict_lstm_Att
testPredictPlot14 = np.empty_like(dataset)
testPredictPlot14[:, :] = np.nan
testPredictPlot14[0:Lgr, :] = x_test_predict_Bilstm_Att

testPredictPlot11g = np.empty_like(dataset)
testPredictPlot11g[:, :] = np.nan
testPredictPlot11g[0:Lgr, :] = x_test_predict_gru
testPredictPlot12g = np.empty_like(dataset)
testPredictPlot12g[:, :] = np.nan
testPredictPlot12g[0:Lgr, :] = x_test_predict_BiGRU
testPredictPlot13g = np.empty_like(dataset)
testPredictPlot13g[:, :] = np.nan
testPredictPlot13g[0:Lgr, :] = x_test_predict_gru_Att
testPredictPlot14g = np.empty_like(dataset)
testPredictPlot14g[:, :] = np.nan
testPredictPlot14g[0:Lgr, :] = x_test_predict_Bigru_Att

plt.figure()
plt.plot(scaler.inverse_transform(dataset[dLgr:]), 'k', label="True Data")
plt.plot(testPredictPlot11[:], label="LSTM Prediction")
plt.plot(testPredictPlot11g[:], label="GRU Prediction")
plt.plot(testPredictPlot12[:], label="BiLSTM Prediction")
plt.plot(testPredictPlot12g[:], label="BiGRU Prediction")
plt.plot(testPredictPlot13[:], label="Att-LSTM Prediction")
plt.plot(testPredictPlot13g[:], label="Att-GRU Prediction")
plt.plot(testPredictPlot14[:], label="Att-BiLSTM Prediction")
plt.plot(testPredictPlot14g[:], label="Att-BiGRU Prediction")

plt.ylabel(WQPu, fontsize=12)
plt.xlabel('Days', fontsize=12)
plt.grid(linestyle=':', linewidth=1)
plt.legend()
plt.savefig('n/'+ WQPn +'_Comp_of_pred'+ l + s + e + w + b + o + d +'.pdf', dpi=300)
plt.close()